# Exercício 1

## Descrição do Problema:
Para este primeiro exercício, foi nos pedido que implementassemos um comunicação privada assíncrona
entre o emissor de uma mensagem (_Emitter_) e o respetivo recetor (_Receiver_).

Para cifrar as mensagens deve-se utilizar uma cifra simétrica segura contra ataques aos "nounces", utilizando a autenticação da mensagem, do criptograma e dos metadados associados a ela; englobando-se, então, na classe __AEAD__ (_Authenticated Encryption with Associated Data_).
Deste modo, o exercício englobará a implementação de duas funções, uma para __cifrar__ a mensagem, e outra,
para __decifrar__.

Adicionalmente, os _"nounces"_ criados devem ser gerados aleatoriamente
para trazer mais segurança à cifra.

No processo deverá occorrer dois tipos de autenticação:

- Do criptograma e metadados, num modo _HMAC_, recorrendo ao uso de "_nounces_";
- Dos participante após receber a mensagem (que se encontrará assinada pelo _sender_).

As chaves utilizadas para cifrar a mensagem (_cipher_key_) e para calcular os códigos 
de autenticação do criptograma e metadados (_mac_key_) são para ser criadas recorrendo ao
protocolo __ECDH__, acordadas entre os agentes.

A assinaturas __ECDSA__ serão no âmbito de autenticar os agentes.

In [1]:
# Todos os imports
import os, json
import asyncio
import random
from pickle import dumps, loads
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, hmac, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# Para a porção de código associada com as queues (na comunicação entre participantes)
import nest_asyncio

nest_asyncio.apply()

## Criação das chaves privadas/públicas

Previamente foi mencionado que iremos desenvolver uma cifra simétrica com autenticação de mensagem e de metadados. 

Para que se possa construir uma cifra simétrica é necessário que os participantes da comunicação conhecem a chave com a qual deverão cifrar/decifrar as mensagens, dado que a mesma será usada para ambas operações. Contudo, a partilha de uma chave é uma operação bastante arriscada, dado que um atacante pode intersetar a mensagem, e passar a ter capacidade de ficar a conhecer todas as mensagens trocadas.  Para mitigar o risco desta operação, iremos utilizar o protocolo **ECDH** para obter duas chaves partilhadas, uma chave utilizada na cifragem da mensagem e outra na autenticação (_cipher_key_ e _mac_key_, respetivamente).

Para que as chaves partilhadas possam ser criadas iremos necessitar que cada participante possua um par de chaves (__privada__ e __pública__), dado que estes são necessárias para o protocolo **ECDH**. A chave __privada__ é gerada sobre uma instância 
da __curva elíptica__, gerando uma chave do tipo _EllipticCurvePrivateKey_,
retirada da biblioteca _cryptography_ e a chave __pública__ é criada a partir da privada.

O método : 
```python 
def generateKeys()
```
cria e retorna estas duas chaves na forma de um __tuplo__ (_private_key, public_key_).

In [2]:
def generateKeys():
    curve = ec.SECP384R1()    
    
    private_key = ec.generate_private_key(curve)
    public_key = private_key.public_key()

    return private_key, public_key #{"private": private_key, "public": public_key}
    

## Assinar mensagem

No enunciado do exercício é indicado que as mensagens devem ser assinadas utilizando o algoritmo **ECDSA**. A assinatura gerada por este permite que o recetor possa verificar que a entidade que enviou a mensagem é o emissor com ele estava a comunicar.

A assinatura digital é obtida através da chave privada de um utilizador e qualquer pessoa que possua a chave pública poderá verificar que a assinatura está associada à chave privada.

O método permite assinar uma dada mensagem :
```python
def signMsg(prv_key, msg)
```
...sendo o atributo _prv_key_ a chave usada para assinar.

In [3]:
def signMsg(prv_key, msg):
    
    signature = prv_key.sign(
    msg,
    ec.ECDSA(hashes.SHA3_256()))
        
    return signature


## Criação das chaves partilhadas

Como mencionamos anteriormente, nesta cifra simétrica iremos utilizar o protocolo **ECDH** para obter uma chave partilhada por ambos os participantes. Para obter esta chave é necessário que cada participante possua uma chave privada e uma chave pública. Este par de chaves foi obtido previamente.

Cada participante deverá enviar ao outro participante duas chaves públicas, uma para cifrar a mensagem e outra para autenticação. Após terem recebido a chave pública do outro participante, será aplicada uma *exchange* entre as chaves. Desta forma obtemos duas chaves partilhadas *cipher\_key* e *mac\_key*

Assim, cria-se o segredo partilhado entre os agentes para cifrar e autenticar os dados.

O método:
```python
def generateShared(prv_cipher, peer_cipher, prv_mac, peer_mac)
```
... utiliza uma chave privada do agente e uma pública do seu _peer_ 
para cada chave.

Neste caso, para cifrar:
- _prv_cipher_ é a chave privada do próprio agente;
- _peer_cipher_ é a chave pública do peer do agente.

Para autenticar:
- _prv_mac_ é a chave privada do próprio agente;
- _peer_mac_ é a chave pública do peer do agente.

... utiliza-se o método _exchange_ para criar a __chave privada__.

Faz-se a derivação da chave para ter mais segurança, destruindo 
alguma possível estrutura que possa existir, ao 
adicionar mais informação à chave.

In [4]:
def generateShared(prv_cipher, peer_cipher, prv_mac, peer_mac):
    
    # Turn a string into a key
    peer_cipher_key = serialization.load_pem_public_key(peer_cipher)
    
    # Create shared key for encrypting
    cipher_key = prv_cipher.exchange(
        ec.ECDH(), peer_cipher_key)
    
    # Perform key derivation for protection
    cipher_derived_key = HKDF(
        algorithm=hashes.SHA3_256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(cipher_key)
    
    # Turn a string into a key 
    peer_mac_key = serialization.load_pem_public_key(peer_mac)
    
    # Create shared key for autentication
    mac_key = prv_mac.exchange(
        ec.ECDH(), peer_mac_key)
    
    # Perform key derivation
    mac_derived_key = HKDF(
        algorithm=hashes.SHA3_256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(mac_key)
    
    return cipher_derived_key, mac_derived_key

## Cifrar mensagem

Neste caso, o método:
```python
def encrypt(msg, cipher_key, mac_key)
```
... utiliza a cifragem autenticada com 
dados associados (AEAD), com uma cifra por blocos 
num modo __GCM__ (_Galois Counter Mode_).
Recorre-se à construção _AES-GCM_ da biblioteca 
_cryptography_, com a chave _cipher_key_ como
seu argumento.

Para o méotodo _encrypt_ é necessário proporcionar 
um _nounce_ e _associated data_. O _nounce_ é óbtido através de um __gerador pseudo aleatório__ 
(PRG), neste caso o com um algoritmo
de _hashing SHA3_256()_. Quanto à _associated data_, esta é gerada 
com um número aleatório de 16 bits.

Por fim, calcula-se o código para autenticação do criptograma
e dos metadados associados, recorrendo a _mac_key_.

O dicionário retornado pelo método inclui:
- Texto cifrado
- Código _HMAC_
- _Nounce_ utilizado
- _Associated data_

In [5]:
def encrypt(msg, cipher_key, mac_key):
    
    digest = hashes.Hash(hashes.SHA3_256())
    nounce = digest.finalize()
    
    ad = os.urandom(16)
    
    aesgcm = AESGCM(cipher_key)
    ct = aesgcm.encrypt(nounce, msg, ad)
    
    h = hmac.HMAC(mac_key, hashes.SHA3_256())
    h.update(ct)
    Hmac = h.finalize()
    
    return {"cipher": ct, "HMAC": Hmac, "nounce": nounce, "ad": ad}

## Decifrar mensagem

Para decifrar, tem-se:
- Verificar se o código _HMAC_ de autenticação;
- Decifrar com o método _decrypt_.

Para verificar se o código _HMAC_ está correto, i.e.
validar a autenticação do agente, é calculado um novo 
código _HMAC_, utilizando a chave _mac_key_ do agente 
que está a decifrar.

No caso de passar a autenticação:

Utiliza o método _decrypt_, com o _nounce_,
_associated data_ e o criptograma a decifrar.

In [6]:
def decrypt(cipher, Hmac, cipher_key, mac_key, nounce, ad):
    
    h = hmac.HMAC(mac_key, hashes.SHA3_256())
    h.update(cipher)
    h_new = h.finalize()
    
    aesgcm = AESGCM(cipher_key)
    
    if h_new == Hmac :
        msg = aesgcm.decrypt(nounce, cipher, ad)
        #print(msg)
    
    return msg

### Funções auxiliares para o Emitter/Receiver

Encontra-se 3 métodos auxiliares para:
- inicialização dos agentes;
- Envio de mensagens;
- Receção de mensagem.

Os métodos para envio e receção de mensagens
têm em consideração o código proporcionado
pela equipa docente, em consideração das 
estruturas _queues_.

O método para __inicialização__:
- Cria os dois pares de chaves pública-privada (para cifrar, autenticar e assinar);
- Serializa as chaves públicas (para enviar ao _peer_);
- Retorna um tuplo com as três chaves e o pacote a enviar ao peer.

...com o nome _init_comm()_.

In [7]:
def init_comm_emitter():
    ## Gerar chaves (cifrar e autenticar + assinar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    prv_mac_key, pub_mac_key = generateKeys()
    prv_sign_key, pub_sign_key = generateKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    ), 
           'mac_key': pub_mac_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    ),
           'sign_key': pub_sign_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    )
            }
    
    return prv_cipher_key, prv_mac_key, prv_sign_key, msg

def init_comm_receiver():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    prv_mac_key, pub_mac_key = generateKeys()
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    ), 
           'mac_key': pub_mac_key.public_bytes(encoding=serialization.Encoding.PEM,
                                       format=serialization.PublicFormat.SubjectPublicKeyInfo
                                    )
            }
    
    return prv_cipher_key, prv_mac_key, msg

async def send(queue, msg):
    
    await asyncio.sleep(random.random())
        
    # put the item in the queue
    await queue.put(msg)
    
    await asyncio.sleep(random.random())
    
async def receive(queue):
    item = await queue.get()

    await asyncio.sleep(random.random())
    aux = loads(item)

    return aux

## Emitter

In [8]:
## Emitter Code
async def emitter(plaintext, queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, prv_mac_key, prv_sign_key, msg = init_comm_emitter()
    
    ## Enviar a chaves públicas para o peer
    await send(queue, dumps(msg))
    print("[E] SENDING PUBLIC KEYS")
    
    ## Receber as chaves públicas do peer
    msg = await receive(queue)
    print("[E] RECEIVED PEER PUBLIC KEYS")
    
    pub_peer_cipher = msg['cipher_key']
    pub_peer_mac = msg['mac_key']
    print("[E] Receiver pub_key_cipher: " +str(msg['cipher_key']))
    print("[E] Receiver pub_key_mac: " +str(msg['mac_key']))
    
    ## Criar as chaves partilhadas (cifrar/autenticar)
    cipher_shared, mac_shared = generateShared(prv_cipher_key, pub_peer_cipher,
                                               prv_mac_key, pub_peer_mac )
    
    ## Cifrar a mensagem
    pkg = encrypt(bytes(plaintext, 'utf-8'), cipher_shared, mac_shared)
    print("[E] MESSAGE ENCRYPTED")
    
    ## Assinar e enviar a mensagem
    pkg_b = dumps(pkg)
    sig = signMsg(prv_sign_key, pkg_b)
    
    # Enviar
    msg_final = {'sig': sig, 'msg': dumps(pkg)}
    
    print("[E] SENDING MESSAGE")
    await send(queue, dumps(msg_final))
    
    print("[E] END")

## Receiver

In [9]:
## Receiver Code
async def receiver(queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, prv_mac_key, msg = init_comm_receiver()
    
    ## Receber as chaves publicas do peer
    pub_keys = await receive(queue)
    
    pub_peer_cipher = pub_keys['cipher_key']
    pub_peer_mac = pub_keys['mac_key']
    pub_peer_sign = pub_keys['sign_key']
    
    print("[R] Emitter pub_key_cipher: " +str(pub_peer_cipher))
    print("[R] Emitter pub_key_mac: " +str(pub_peer_mac))
    
    ## Gerar shared keys
    cipher_shared, mac_shared = generateShared(prv_cipher_key, pub_peer_cipher,
                                               prv_mac_key, pub_peer_mac)

    ## Enviar as chaves públicas ao peer
    await send(queue, dumps(msg))
    print("[R] AWAIT CIPHER")
    ciphertext = await receive(queue)
    print("[R] CIPHER RECEIVED")
    
    ## Receber a mensagem (Assinatura)
    peer_sign_key = serialization.load_pem_public_key(pub_peer_sign)
    
    ## Validar a correçaõ da assinatura
    peer_sign_key.verify(ciphertext['sig'], ciphertext['msg'], ec.ECDSA(hashes.SHA3_256()))
       
    msg_dict = loads(ciphertext['msg'])
                      
    ## Decifrar essa mensagem       
    plain_text = decrypt(msg_dict['cipher'],msg_dict['HMAC'], cipher_shared, mac_shared, msg_dict['nounce'], msg_dict['ad'])
    
    ## Apresentar no terminal
    print("[R] Plaintext: " + plain_text.decode('utf-8'))
       

### Função "main" para teste da comunicação

In [10]:
def test(msg):
    loop = asyncio.get_event_loop()
    queue = asyncio.Queue(10)
    asyncio.ensure_future(emitter(msg, queue), loop=loop)
    loop.run_until_complete(receiver(queue))
    # emitter(msg)

test("HELLO WORLD!")

[R] Emitter pub_key_cipher: b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAEBxmtKmMScCrmVT+SiiNyyX1N4gzut5bO\n9CdLH8DO5DSYZOrGlBUofkwWv4APyMMhwXy0c3x8CMEiKu54ENlW/yPWEv5JZx4B\nA7IVWtS0L5gRHtZjBNFcydfMqQL0Y/RG\n-----END PUBLIC KEY-----\n'
[R] Emitter pub_key_mac: b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAENAJIWqgdbvxqMQqQqWSxCni+LCCJLXfC\nFKxL9tMCrtyW6a0CpKwuykG0OyX9g+3Sg+eOzTE9fucjc3yEKBKD/Hl+eBZpNniK\nEMvI6T/Eq3aF2Id2BNs3IPbHJHKnvL9u\n-----END PUBLIC KEY-----\n'
[E] SENDING PUBLIC KEYS
[R] AWAIT CIPHER
[E] RECEIVED PEER PUBLIC KEYS
[E] Receiver pub_key_cipher: b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAEeUG9vh22YDNxGYiaJFGp9FJTt6qDK5wr\nrbtWjC9Lt4XAK28bsYOoJHChWo+W8ZivvpE4Iw9iKpbTDb2QGQaBXQesQrvTXCMR\nML+Evh6d+wDGP9B1wkUlr1L9AXM9MECi\n-----END PUBLIC KEY-----\n'
[E] Receiver pub_key_mac: b'-----BEGIN PUBLIC KEY-----\nMHYwEAYHKoZIzj0CAQYFK4EEACIDYgAE12wFWVpfV9ET0H2GPeWDFKyAIdEw/tXc\n8gGApoZln8x7v6P426RWrBrhUR7C5THPAfm18i0jeof9PB8I1XV1V